In [1]:
from gensim.models import doc2vec
from gensim.models.doc2vec import LabeledSentence
import gensim
from collections import namedtuple
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.models import model_from_json
from keras.models import load_model
import random
from keras import optimizers
import re
import gensim.models as g
import logging
import numpy as np
import codecs

C:\Anaconda2\envs\py35\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


Some necessary links of data

In [14]:
chatbot_data='Dataset\\data.json_responses.json'#link leads to the file containing data for chatbot
doc2vec_data='Dataset\data11_9_.txt'#link leads to the file containing data for doc2vec model, I named it as data11_9_ according to the date I made this data set

Import data for chatbot

In [3]:
import json
with open(chatbot_data,encoding='utf8') as json_data:
    intents = json.load(json_data)
content=intents['intents']

Create one hot vector for each class (we have 114 classes in total)

In [4]:
one_hot=np.zeros((len(content),len(content)))
one_hot[range(len(one_hot)),range(len(one_hot))]=1

Convert rough data which is text type into one hot encoding

In [5]:
rough_text=[]
Y=[]
for i in range(len(content)):
    content[i]['patterns']
    l=content[i]['patterns']
    for j in range(len(l)):
        Y.append(one_hot[i])
        rough_text.append(l[j].lower())

# Create para2vec model

Prepare some necessary stuff

In [6]:
#doc2vec parameters
vector_size = 120
window_size = 4
min_count = 1
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 150
dm = 0
worker_count = 1 #number of parallel processes

def trans(content):#convert content into taggedDocument type
    result=[]
    analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
    for a,b in enumerate(content):
        words = b.lower().split()
        tags = [a]
        result.append(gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(b), [a]))
    return result

p2v_content=[]
with open(doc2vec_data,encoding='utf8') as f:
    for line in f:
        p2v_content.append(line)
p2v_content.extend(rough_text)
utf8_content=trans(p2v_content)
model = gensim.models.doc2vec.Doc2Vec(size=vector_size, window=window_size, min_count=min_count,seed=0, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1,  iter=train_epoch)
model.build_vocab(utf8_content)

Create doc2vec model

In [7]:
%time model.train(utf8_content, total_examples=model.corpus_count, epochs=model.iter)

Wall time: 3.01 s


246036

# Create a "brain" for chatbot

In [8]:
def create_model(X,Y):
    seed = 7
    np.random.seed(seed)
    # create model
    model = Sequential()
    model.add(Dense(100, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(300, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(340, activation='relu'))
    model.add(Dense(390, activation='relu'))
    model.add(Dense(Y.shape[1], activation='softmax'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X, Y, epochs=1200, batch_size=10,shuffle=True)
    return model

Get vectors to feed neuronNet from texts via doc2vec model

In [9]:
X=[]    
for i in range(len(rough_text)):
    model.random.seed(7)
    vec=model.infer_vector(re.sub('\?','',rough_text[i]).lower().split(' '))
    X.append(vec)
Y=np.array(Y)
X=np.array(X)
print (X.shape, Y.shape)

(1888, 120) (1888, 110)


Create neuron network model

In [10]:
neu_model=create_model(X,Y)

Epoch 1/1
1888/1888 [==============================] - 2s - loss: 0.0090 - acc: 0.0185     


Prediction step for forming answer for users

In [11]:
def predictions(quest):
    model.random.seed(7)
    vec=np.array(model.infer_vector(quest.split(' '))).reshape(1,-1)
    out_put=neu_model.predict(vec)
    return content[np.argmax(out_put)]

In [12]:
def get_answer(quest):
    answer=predictions(quest)#this is a list containing a set of answers
    a=random.randint(0,len(answer)-1)
    return answer['responses'][a]

Input your question into the blank in the function below and it will return an answer

In [13]:
get_answer("chào")

'có em đây chuyện gì rồi sẽ qua thôi'